In [1]:
from dotenv import load_dotenv
from openai import OpenAI

In [2]:
load_dotenv()
client = OpenAI()

In [12]:
question = input("사용자 요청 메시지")
response = client.chat.completions.create(
  model="gpt-4.1-mini",
  messages = [
    # AI 가 할 일
    {'role' : "system", "content" : "You are a helpful assistant. using locale language."},
    # 사용자 요청
    {'role' : "user", "content" : question}
  ],
)
print(f'AI : {response.choices[0].message.content}')

AI : 오늘 점심으로 김치찌개 어떠세요? 따뜻하고 얼큰한 맛이 입맛 돋우기에 좋아요. 밥과 함께 먹으면 든든한 한 끼가 될 거예요!


In [30]:
# 실시간 데이터는 LLM 의 학습 내용이 아닙니다.
# ㄴ 함수 호출 또는 tools 사용해서 답변을 할 수 있는 기능이 있습니다.

# 시간 구하기 함수
from datetime import datetime

def get_crrent_time():
    now = datetime.now().strftime('%H:%M:%S')
    print(f'= log : {now} =')
    return now

get_crrent_time()
# 날짜 구하기 함수 : 예정 '%Y:%m:%d'
def get_crrent_date():
    now = datetime.now().strftime('%Y:%m:%d')
    print(f'= log : {now} =')
    return now

# gpt 가 사용할 함수 목록을 정의
myfunctions = [
    {
        "name" : "get_crrent_time",
        "description" : "현재 시간 출력합니다. 포맷 HH:MM:SS 입니다."
    },
    {
        "name" : "get_crrent_date",
        "description" : "현재 날짜 출력합니다. 포맷 YYYY년 MM월 DD일 입니다."
    }
]

   

= log : 12:02:41 =


In [33]:
question = '오늘 날짜를 알려줘'
response = client.chat.completions.create(
  model="gpt-4.1-mini",
  messages = [
    {'role' : "system", "content" : "You are a helpful assistant. using locale language."},
    {'role' : "user", "content" : question}
  ],
  functions=myfunctions
)
# response 는 ChatCompletions 타입 객체. 그안에 속성은 객체 타입 or 문자열
print(f'log response : {response.model_dump_json(indent=2)}')

log response : {
  "id": "chatcmpl-CYkZW4ndJdk8f737Qs8kGEQ28YuXU",
  "choices": [
    {
      "finish_reason": "function_call",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": null,
        "refusal": null,
        "role": "assistant",
        "annotations": [],
        "audio": null,
        "function_call": {
          "arguments": "{}",
          "name": "get_crrent_date"
        },
        "tool_calls": null
      }
    }
  ],
  "created": 1762398210,
  "model": "gpt-4.1-mini-2025-04-14",
  "object": "chat.completion",
  "service_tier": "default",
  "system_fingerprint": "fp_4c2851f862",
  "usage": {
    "completion_tokens": 12,
    "prompt_tokens": 89,
    "total_tokens": 101,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio_tokens": 0,
      "reasoning_tokens": 0,
      "rejected_prediction_tokens": 0
    },
    "prompt_tokens_details": {
      "audio_tokens": 0,
      "cached_tokens": 0
    }
  }
}


In [34]:
# 함수 호출을 하고 답변을 만드는 추가적인 요청이 필요합니다.
import json
fn_name = response.choices[0].message.function_call.name
# ↪ log : FunctionCall(arguments='{}', name='get_crrent_time')
# fn_name = getattr(response.choices[0].message.function_call, 'name',)
print(f'log : {fn_name}')
if fn_name:
  func_response = globals()[fn_name]() # 문자열로 된 함수를 호출하는 방법
  followup_response = client.chat.completions.create(
    model='gpt-4.1-mini',
    messages = [
    {'role' : "system", "content" : "You are a helpful assistant. using locale language."},
    {'role' : "user", "content" : f'{fn_name} 함수를 실행한 결과 {func_response} 이용하여 최종 응답 만들어줘.'}
  ],
  functions=myfunctions
  )
  print(followup_response.model_dump_json(indent=2))
  # 함수 실행한 두번째 요청 응답
  result = followup_response.choices[0].message.content
else:
  # 첫번째 요청 응답
  result = response.choices[0].message.content

print(f'AI : {result}')

log : get_crrent_date
= log : 2025:11:06 =


{
  "id": "chatcmpl-CYkZXpipzgPmj3bdm44Y9LI24tWHW",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "오늘 날짜는 2025년 11월 6일입니다. 도움이 필요하시면 언제든 말씀해 주세요!",
        "refusal": null,
        "role": "assistant",
        "annotations": [],
        "audio": null,
        "function_call": null,
        "tool_calls": null
      }
    }
  ],
  "created": 1762398211,
  "model": "gpt-4.1-mini-2025-04-14",
  "object": "chat.completion",
  "service_tier": "default",
  "system_fingerprint": "fp_4c2851f862",
  "usage": {
    "completion_tokens": 26,
    "prompt_tokens": 109,
    "total_tokens": 135,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio_tokens": 0,
      "reasoning_tokens": 0,
      "rejected_prediction_tokens": 0
    },
    "prompt_tokens_details": {
      "audio_tokens": 0,
      "cached_tokens": 0
    }
  }
}
AI : 오늘 날짜는 2025년 11월 6일입니다. 도움이 필요하시면 언제든 말씀해 주세요!
